<p align="center"><img src="logo/graphx.png" alt="GraphX Logo" width="500"/></p>
# **Lab 4 - GraphX**
#### The following steps demonstrate how to use GraphX, the graph processing component in Spark. In this notebook you will see how to make a base graph and apply functions to it.

Throughout this tutorial we will use the following property graph that makes a small social network with users and their ages modeled as vertices and likes modeled as directed edges.

<img src="files/logo/social_graph.png" width="500">

We begin by creating the property graph from arrays of vertices and edges. Later we will demonstrate how to load real data. Here we use the `Edge` class. Edges have a `srcId` and a `dstId` corresponding to the source and destination vertex identifiers. In addition, the `Edge` class has an attr member that stores the edge property (in this case the number of likes). Use `sc.parallelize` to construct the following RDDs from the `vertexArray` and `edgeArray` variables, and then build a property graph. The basic property graph constructor takes an RDD of vertices (with type `RDD[(VertexId, V)]`) and an RDD of edges (with type `RDD[Edge[E]]`) and builds a graph (with type `Graph[V, E]`). 

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
 
val vertexArray = Array(
  (1L, ("Alice", 28)),
  (2L, ("Bob", 27)),
  (3L, ("Charlie", 65)),
  (4L, ("David", 42)),
  (5L, ("Ed", 55)),
  (6L, ("Fran", 50))
  )
val edgeArray = Array(
  Edge(2L, 1L, 7),
  Edge(2L, 4L, 2),
  Edge(3L, 2L, 4),
  Edge(3L, 6L, 3),
  Edge(4L, 1L, 1),
  Edge(5L, 2L, 2),
  Edge(5L, 3L, 8),
  Edge(5L, 6L, 3)
  )

val vertexRDD: RDD[(Long, (String, Int))] = sc.parallelize(vertexArray)
val edgeRDD: RDD[Edge[Int]] = sc.parallelize(edgeArray)
val graph: Graph[(String, Int), Int] = <FILL IN>

In many cases we will want to extract the vertex and edge RDD views of a graph. As a consequence, the graph class contains members (`graph.vertices` and `graph.edges`) to access the vertices and edges of the graph. While these members extend `RDD[(VertexId, V)`] and `RDD[Edge[E]]` they are actually backed by optimized representations that leverage the internal GraphX representation of graph data. Below, use `graph.vertices` to display the names of the users that are at least 30 years old. The output should contain:
```
David is 42
Fran is 50
Ed is 55
Charlie is 65
```

In [ ]:
graph.vertices.<FILL IN>.collect.foreach(v => println(s"${v._2._1} is ${v._2._2}"))

Now, use the `graph.triplets` view to display who likes who. The output should look like:
```
Bob likes Alice
Bob likes David
Charlie likes Bob
Charlie likes Fran
David likes Alice
Ed likes Bob
Ed likes Charlie
Ed likes Fran
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
for (triplet <- <FILL IN>) {
  println(s"${triplet.srcAttr._1} likes ${triplet.dstAttr._1}")
}

If someone likes someone else more than 5 times than that relationship is getting pretty serious. For extra credit, find the lovers.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
for (triplet <- <FILL IN>) {
  println(s"${triplet.srcAttr._1} loves ${triplet.dstAttr._1}")
}

You can compute the in-degree of the graph using the `graph.inDegrees` operators that returns a `VertexRDD[Int]`.

In [ ]:
val inDegrees: VertexRDD[Int] = graph.inDegrees

Now, let's incorporate the in and out degree of each vertex into the vertex property. To do this, we first define a `User` class to better organize the vertex property and build a new graph with the user property. We initialized each vertex with 0 in and out degree. Then, we join the in and out degree information with each vertex building the new vertex property. Here we use the `outerJoinVertices` method of `Graph` that takes two argument lists: (i) an RDD of vertex values, and (ii) a function from the id, attribute, and Optional matching value in the RDD to a new vertex value. The `outerJoinVertices` has the following type signature:
```scala
def outerJoinVertices[U, VD2](other: RDD[(VertexID, U)])
    (mapFunc: (VertexID, VD, Option[U]) => VD2): Graph[VD2, ED]
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

// Define a class to more clearly model the user property
case class User(name: String, age: Int, inDeg: Int, outDeg: Int)

// Create a user Graph
val initialUserGraph: Graph[User, Int] = graph.mapVertices{ case (id, (name, age)) => User(name, age, 0, 0) }

// Fill in the degree information
val userGraph = initialUserGraph.outerJoinVertices(initialUserGraph.inDegrees) {
  case (id, u, inDegOpt) => <FILL IN>
}.<FILL IN> {
  case (id, u, outDegOpt) => <FILL IN>
}


Using the `degreeGraph` print the number of people who like each user:
```
User 1 is called Alice and is liked by 2 people.
User 2 is called Bob and is liked by 2 people.
User 3 is called Charlie and is liked by 1 people.
User 4 is called David and is liked by 1 people.
User 5 is called Ed and is liked by 0 people.
User 6 is called Fran and is liked by 2 people.
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
for ((id, property) <- userGraph.vertices.collect) {
  println(s"User $id is called <FILL IN> and is liked by ${property.inDeg} people.")
}

Print the names of the users who are liked by the same number of people they like.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
userGraph.vertices.filter {
  case (id, u) => <FILL IN>
}.collect.foreach {
  case (id, property) => println(property.name)
}

Suppose we want to find the oldest follower of each user, using the above property graph. The `aggregateMessages` operator allows us to do this. This operator applies a user defined `sendMsg` function to each edge triplet in the graph and then uses the `mergeMsg` function to aggregate those messages at their destination vertex.
```scala
class Graph[VD, ED] {
  def aggregateMessages[Msg: ClassTag](
      sendMsg: EdgeContext[VD, ED, Msg] => Unit,
      mergeMsg: (Msg, Msg) => Msg,
      tripletFields: TripletFields = TripletFields.All)
    : VertexRDD[Msg]
}
```
Bellow, you can find the oldest follower for each user by sending a message containing the name and age of each follower and aggregating the messages by taking the message from the older follower.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
// Find the oldest follower for each user
val oldestFollower: VertexRDD[(String, Int)] = userGraph.aggregateMessages[(String, Int)](
  // sendMsg
  triplet => <FILL IN>,
  // mergeMsg
  (a, b) => <FILL IN>
)


Display the oldest follower for each user as bellow. Note that some users may have no messages.
```
David is the oldest follower of Alice.
Charlie is the oldest follower of Bob.
Ed is the oldest follower of Charlie.
Bob is the oldest follower of David.
Ed does not have any followers.
Charlie is the oldest follower of Fran.
```

In [ ]:
userGraph.vertices.leftJoin(oldestFollower) { (id, user, optOldestFollower) =>
  optOldestFollower match {
    case None => s"${user.name} does not have any followers."
    case Some((name, age)) => s"${name} is the oldest follower of ${user.name}."
  }
}.collect.foreach { case (id, str) => println(str) }

Now, find the average follower age of the followers of each user.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
val averageAge: VertexRDD[Double] = userGraph.aggregateMessages[(Int, Double)](
  // map function returns a tuple of (1, Age)
  triplet => <FILL IN>,
  // reduce function combines (sumOfFollowers, sumOfAge)
  (a, b) => <FILL IN>
).mapValues((id, p) => p._2 / p._1)

// Display the results
userGraph.vertices.leftJoin(averageAge) { (id, user, optAverageAge) =>
  optAverageAge match {
    case None => s"${user.name} does not have any followers."
    case Some(avgAge) => s"The average age of ${user.name}\'s followers is $avgAge."
  }
}.collect.foreach { case (id, str) => println(str) }

Suppose you want to study the community structure of users that are 30 or older. To support this type of analysis GraphX includes the `subgraph` operator that takes vertex and edge predicates and returns the graph containing only the vertices that satisfy the vertex predicate (evaluate to true) and edges that satisfy the edge predicate and connect vertices that satisfy the vertex predicate. 

In the following example, restrict your graph to the users that are 30 or older, and  examine the communities in this restricted graph. Connected components are labeled (numbered) by the lowest vertex Id in that component. Notice that by examining the subgraph you have disconnected David from the rest of his community. Moreover his connections to the rest of the graph are through younger users.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code
val olderGraph = userGraph.subgraph(<FILL IN>)

// compute the connected components
val cc = olderGraph.connectedComponents

// display the component id of each user:
olderGraph.vertices.leftJoin(cc.vertices) {
  case (id, user, comp) => s"${user.name} is in component ${comp.get}"
}.collect.foreach{ case (id, str) => println(str) }